In [1]:
pip install mediapipe opencv-python numpy


Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import mediapipe as mp
import numpy as np
import subprocess

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Function to set volume using osascript (AppleScript)
def set_volume(volume):
    volume = max(0, min(100, volume))  # Ensure volume stays within 0-100
    script = f'set volume output volume {int(volume)}'
    subprocess.run(["osascript", "-e", script])

# Start Video Capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert BGR to RGB for MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get landmarks for thumb (tip) and index finger (tip)
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

            # Convert to pixel coordinates
            h, w, _ = frame.shape
            x1, y1 = int(thumb_tip.x * w), int(thumb_tip.y * h)
            x2, y2 = int(index_tip.x * w), int(index_tip.y * h)

            # Calculate distance between thumb and index finger
            distance = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

            # Map distance (scaled to 0-100 for volume)
            volume = np.interp(distance, [20, 200], [0, 100])

            # Set system volume
            set_volume(volume)

            # Display Volume on Screen
            cv2.putText(frame, f"Volume: {int(volume)}%", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show Camera Feed
    cv2.imshow("Hand Gesture Volume Control", frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1751909849.460283   47903 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1751909849.485638   48153 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751909849.495580   48153 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


KeyboardInterrupt: 